In [1]:
import pandas as pd 
import numpy as np

In [27]:
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
df.shape

(20800, 5)

In [29]:
df.dropna(inplace=True)

In [30]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [31]:
X = df.drop(columns=['label'],axis=1)
y = df['label']

In [32]:
X.shape,y.shape

((18285, 4), (18285,))

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk 
import re
from nltk.corpus import stopwords

In [34]:
messages = X.copy()

In [38]:
messages.reset_index(inplace=True)

In [40]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [41]:
corpus[0] # Cleaned Corpus

'hous dem aid even see comey letter jason chaffetz tweet'

In [44]:
voc_size = 5000  # Define Vocab size
onehotrep = [one_hot(words,voc_size) for words in corpus]

In [50]:
# Apply pad sequence : to SAME length
set_length = 60
embedded_docs = pad_sequences(onehotrep,padding="pre",maxlen=set_length)
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 4315, 3539,
       1957, 2340,  737,  561, 1027], dtype=int32)

In [57]:
# Creating Model : 

embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=set_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 60, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Model is ready to fit

In [62]:
# Let X and y ready 
X = np.array(embedded_docs)
y = np.array(y)
X.shape,y.shape

((18285, 60), (18285,))

In [64]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=33)

In [66]:
X_train.shape,y_train.shape

((12250, 60), (12250,))

In [71]:
# MOdel training:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0207 - accuracy: 0.9944 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 11s 59ms/step - loss: 0.0140 - accuracy: 0.9963 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0091 - accuracy: 0.9980 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0064 - accuracy: 0.9984 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
192/192 [==============================] - 11s 59ms/step - loss: 0.0029 - accuracy: 0.9994 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
192/192 [==============================] - 11s 58ms/step - loss: 0.0017 - accu

In [73]:
ypred=model.predict_classes(X_test)

In [75]:
from sklearn.metrics import confusion_matrix

In [77]:
confusion_matrix(y_test,ypred)

array([[3129,  333],
       [ 246, 2327]])

In [79]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,ypred)

0.904059652029826